In [31]:
import tensorflow as tf
import tensorflow.keras as keras
import random

In [32]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

if gpus:
  try:    
    for gpu in gpus:
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except Exception as e:
    print("EXCEPTION")
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
EXCEPTION
Visible devices cannot be modified after being initialized


In [33]:
val_size = 0.3
rand_seed = random.randint(0, 2**32 - 1)
img_dim = 64
batch_size = 16

In [34]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "data/training",
  validation_split = val_size,
  subset="training",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  "data/training",
  validation_split = val_size,
  subset = "validation",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size)

Found 1970 files belonging to 2 classes.
Using 1379 files for training.
Found 1970 files belonging to 2 classes.
Using 591 files for validation.


In [35]:
image_input = keras.Input(shape = (img_dim, img_dim, 3))
x = keras.layers.Rescaling(1.0 / 255)(image_input)
x = keras.layers.Conv2D(192, kernel_size = 2, activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Conv2D(192, kernel_size = 2, activation = 'relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dropout(0.05)(x)
x = keras.layers.Flatten()(x)
feature_output = keras.layers.Dense(1024, activation = 'relu')(x)
classification_output = keras.layers.Dense(2)(feature_output)

model = keras.models.Model([image_input], classification_output)

In [36]:
opt = keras.optimizers.Adagrad(learning_rate = 0.01)
model.compile(optimizer = opt,
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 63, 63, 192)    │         2,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 30, 30, 192)    │       147,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 15, 15, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 15, 15, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 43200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │    44,237,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,390,018 (169.33 MB)

 Trainable params: 44,390,018 (169.33 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
epochs = 32

In [38]:
hist = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[]
)

Epoch 1/32


/home/kilnaar/anaconda3/envs/ai570/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_25']
Received: inputs=Tensor(shape=(None, 64, 64, 3))
  warnings.warn(msg)


87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6379 - loss: 0.6956 - val_accuracy: 0.9577 - val_loss: 0.2884
Epoch 2/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8315 - loss: 0.4038 - val_accuracy: 0.9679 - val_loss: 0.1428
Epoch 3/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9411 - loss: 0.1756 - val_accuracy: 0.9255 - val_loss: 0.1892
Epoch 4/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9333 - loss: 0.1753 - val_accuracy: 0.9662 - val_loss: 0.1121
Epoch 5/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9652 - loss: 0.1011 - val_accuracy: 0.9797 - val_loss: 0.0784
Epoch 6/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9839 - loss: 0.0619 - val_accuracy: 0.7665 - val_loss: 0.4779
Epoch 7/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9696 - loss: 0.0840 - val_accuracy: 0.9780 - val_loss: 0.0796
Epoch 8/32
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9869 - loss: 0.0429 - val_accuracy: 0.9831 - val_loss: 0.0634
Ep

In [39]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  "data/test",
  shuffle = False,
  image_size = (img_dim, img_dim)
)


Found 30 files belonging to 2 classes.


In [40]:
model.evaluate(test_ds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 1.0000 - loss: 0.0170


[0.017005592584609985, 1.0]